In [50]:

!pip install torch==2.4.1 torchvision==0.19.1 --extra-index-url https://download.pytorch.org/whl/cu124



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11936.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124


In [51]:
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
11945.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-1i280pnh
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-1i280pnh
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit 19cb3670caea21e11b35e01ce1115a4a35b27fb5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached SAM_2-1.0-cp311-cp311-linux_x86_64.whl


In [1]:
from PIL import Image
from lang_sam import LangSAM

model = LangSAM()
image_pil = Image.open("/home/abka03/Projects/xl-vlms/playground/bus.jpg").convert("RGB")
text_prompt = "bus"
results = model.predict([image_pil], [text_prompt])

/home/abka03/.conda/envs/lang_sam/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Predicting 1 masks
Predicted 1 masks


/home/abka03/.conda/envs/lang_sam/lib/python3.11/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [15]:
import os
import torch
import numpy as np
from PIL import Image, ImageOps
from transformers import BlipProcessor, BlipForQuestionAnswering
from lang_sam import LangSAM


device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
vqa_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)


def get_all_objects_vqa(image_pil):
    """Asks multiple questions to BLIP VQA to extract all possible objects in the image."""
    questions = [
        "What are the objects in the image?",
        "List all the things present.",
        "What items do you see?",
        "Describe everything in the image.",
        "What is inside this picture?"
    ]

    detected_objects = set()  # Store unique objects

    for question in questions:
        inputs = processor(image_pil, question, return_tensors="pt").to(device)
        with torch.no_grad():
            out = vqa_model.generate(**inputs, max_length=500)
            answer = processor.decode(out[0], skip_special_tokens=True)
            detected_objects.update(answer.split(","))  # Handle multiple objects

    return list(detected_objects)  # Convert to list

def vqa_and_segment(image_path):
    # Load the image
    image_pil = Image.open(image_path).convert("RGB")
    image_np = np.array(image_pil)  # Convert image to numpy for processing
    # Load the VQA model and processor


    # Get the VQA model's answers
     # Get a list of all objects in the image using multiple VQA questions
    object_list = get_all_objects_vqa(image_pil)
    print("Detected objects:", object_list)

    # Initialize LangSAM for segmentation
    langsam_model = LangSAM()


    # Segment the image based on the VQA answers
    segmented_images = []
    for  answer in object_list:
        results = langsam_model.predict([image_pil], [answer])[0]
        masks = results["masks"]
        labels = results["labels"]
        bboxes = results["boxes"]  # Bounding boxes in (x1, y1, x2, y2) format

        for i, (mask, label, bbox) in enumerate(zip(masks, labels, bboxes)):
            x1, y1, x2, y2 = map(int, bbox)  # Convert bounding box coordinates to integers

            # Convert mask to binary (0 or 255)
            mask_np = (mask * 255).astype(np.uint8)

            # Extract the bounding box region from the image and mask
            cropped_image_np = image_np[y1:y2, x1:x2]
            cropped_mask_np = mask_np[y1:y2, x1:x2]

            # Apply mask to keep only the segmented object
            segmented_np = np.zeros_like(cropped_image_np)
            for c in range(3):  # Apply mask to all RGB channels
                segmented_np[:, :, c] = cropped_image_np[:, :, c] * (cropped_mask_np // 255)

            segmented_pil = Image.fromarray(segmented_np)
            #segmented_pil = ImageOps.expand(segmented_pil, border=10, fill="white")  # Optional padding
            segmented_images.append((segmented_pil, label))
            # Save the cropped segmented image
    return segmented_images  # Return list of (PIL image, label)

# Example usage
image_path = "/home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000009.jpg"

segments = vqa_and_segment(image_path)
save_path = "/home/abka03/Projects/xl-vlms/playground/datasets/segmention"
# Display and print results
for i, (segment_img, label) in enumerate(segments):
    segment_filename = os.path.join(save_path, f"{label}_{i}.png")
    segment_img.save(segment_filename)
    #segment_img.show()  # Display each cropped segment
    print(f"Segment {i}: Label = {label}")


Detected objects: ['lunch', 'food']
Predicting 1 masks
Predicted 1 masks
Predicting 1 masks
Predicted 1 masks
Segment 0: Label = lunch
Segment 1: Label = lunch
Segment 2: Label = lunch
Segment 3: Label = food
Segment 4: Label = food
Segment 5: Label = food
Segment 6: Label = food
Segment 7: Label = food
Segment 8: Label = food
Segment 9: Label = food
Segment 10: Label = food


In [12]:
import os
import torch
import numpy as np
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
from lang_sam import LangSAM

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load models once
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
vqa_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
langsam_model = LangSAM()

def get_all_objects_vqa(image_pil):
    """Extracts a list of objects from an image using VQA."""
    questions = [
        "What are the objects in the image?",
        "List all the things present.",
        "What items do you see?",
        "Describe everything in the image.",
        "What is inside this picture?"
    ]

    detected_objects = set()  # Use a set to store unique objects

    for question in questions:
        inputs = processor(image_pil, question, return_tensors="pt").to(device)
        with torch.no_grad():
            out = vqa_model.generate(**inputs, max_length=500)
            answer = processor.decode(out[0], skip_special_tokens=True)
            detected_objects.update(obj.lower().strip() for obj in answer.split(","))  # Normalize text

    return list(detected_objects)

def vqa_and_segment(image_path):
    """Processes an image, extracts objects, and segments them as a batch. Returns a list of (image, label, index)."""
    try:
        # Load and convert the image
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)

        # Get detected objects using VQA
        object_list = get_all_objects_vqa(image_pil)
        print(f"Detected objects in {image_path}: {object_list}")

        # Batch processing: Predict segmentation for all detected objects at once
        results = langsam_model.predict([image_pil], object_list)[0]
        masks, labels, bboxes = results["masks"], results["labels"], results["boxes"]

        segmented_results = []
        for i, (mask, label, bbox) in enumerate(zip(masks, labels, bboxes)):
            x1, y1, x2, y2 = map(int, bbox)

            # Convert mask to binary format
            mask_np = (mask * 255).astype(np.uint8)

            # Extract relevant region
            cropped_image_np = image_np[y1:y2, x1:x2]
            cropped_mask_np = mask_np[y1:y2, x1:x2]

            # Apply mask to retain segmented object
            segmented_np = np.zeros_like(cropped_image_np)
            for c in range(3):  # Apply mask to all RGB channels
                segmented_np[:, :, c] = cropped_image_np[:, :, c] * (cropped_mask_np // 255)

            segmented_pil = Image.fromarray(segmented_np)
            segmented_results.append((segmented_pil, label, i))

        return segmented_results  # List of (PIL image, label, index)

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return []

def process_images_in_folder(folder_path):
    """Processes all images in a folder and returns segmentation results."""
    if not os.path.exists(folder_path):
        print("Error: Folder does not exist.")
        return {}

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No images found in the folder.")
        return {}

    print(f"Processing {len(image_files)} images from {folder_path}...")

    all_segments = {}
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_segments[image_path] = vqa_and_segment(image_path)

    return all_segments  # Dictionary {image_path: [(image, label, index), ...]}

# Example usage
image_folder = "/home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train"
save_folder = "/home/abka03/Projects/xl-vlms/playground/dataset/segments"

segmentation_results = process_images_in_folder(image_folder)

# Save segmented images outside the function
os.makedirs(save_folder, exist_ok=True)
for image_path, segments in segmentation_results.items():
    for segment_img, label, index in segments:
        filename = f"{label}_{index}.png"
        save_path = os.path.join(save_folder, filename)
        segment_img.save(save_path)
        print(f"Saved: {save_path}")


Processing 4 images from /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train...
Detected objects in /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000030.jpg: ['vase', 'flowers']
Error processing /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000030.jpg: The expanded size of the tensor (8) must match the existing size (4) at non-singleton dimension 2.  Target sizes: [4, 19947, 8].  Tensor sizes: [8, 1, 4]
Detected objects in /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000009.jpg: ['lunch', 'food']
Error processing /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000009.jpg: The expanded size of the tensor (8) must match the existing size (4) at non-singleton dimension 2.  Target sizes: [4, 17821, 8].  Tensor sizes: [8, 1, 4]
Detected objects in /home/abka03/Projects/xl-vlms/playground/datasets/coco8/images/train/000000000025.jpg: ['giraffe', 'trees']
Error p